setup

In [9]:
!pip install pycryptodome

In [10]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import time
import os

key generation and storage

In [11]:
def generate_aes_key(key_length):
    key = get_random_bytes(key_length // 8)
    with open(f'aes_key_{key_length}.key', 'wb') as f:
        f.write(key)
    return key

def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    with open('private.pem', 'wb') as f:
        f.write(private_key)
    with open('public.pem', 'wb') as f:
        f.write(public_key)
    return key

def load_aes_key(key_length):
    with open(f'aes_key_{key_length}.key', 'rb') as f:
        key = f.read()
    return key

def load_rsa_keys():
    with open('private.pem', 'rb') as f:
        private_key = RSA.import_key(f.read())
    with open('public.pem', 'rb') as f:
        public_key = RSA.import_key(f.read())
    return private_key, public_key

AES Encryption/Decryption

In [12]:
def aes_encrypt(data, key, mode):
    cipher = AES.new(key, mode)
    ciphertext = cipher.encrypt(data)
    with open('aes_encrypted.bin', 'wb') as f:
        f.write(cipher.iv + ciphertext)
    return cipher.iv + ciphertext

def aes_decrypt(ciphertext, key, mode):
    iv = ciphertext[:16]
    cipher = AES.new(key, mode, iv=iv)
    plaintext = cipher.decrypt(ciphertext[16:])
    return plaintext

def pad(data):
    while len(data) % 16 != 0:
        data += b' '
    return data

RSA Encryption/Decryption and Signing/Verification

In [13]:
def rsa_encrypt(data, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    ciphertext = cipher.encrypt(data)
    with open('rsa_encrypted.bin', 'wb') as f:
        f.write(ciphertext)
    return ciphertext

def rsa_decrypt(ciphertext, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext

def rsa_sign(data, private_key):
    h = SHA256.new(data)
    signature = pkcs1_15.new(private_key).sign(h)
    with open('rsa_signature.sig', 'wb') as f:
        f.write(signature)
    return signature

def rsa_verify(data, signature, public_key):
    h = SHA256.new(data)
    try:
        pkcs1_15.new(public_key).verify(h, signature)
        print("Signature is valid.")
    except (ValueError, TypeError):
        print("Signature is invalid.")

SHA-256 Hashing

In [14]:
def sha256_hash(data):
    h = SHA256.new(data)
    print(h.hexdigest())

Timing Functionality

In [15]:
def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    return result

Command Line Interface

In [16]:
def main():
    while True:
        print("Choose an option:")
        print("1. AES Encryption")
        print("2. AES Decryption")
        print("3. RSA Encryption")
        print("4. RSA Decryption")
        print("5. RSA Signature")
        print("6. RSA Verification")
        print("7. SHA-256 Hashing")
        print("8. Exit")

        choice = int(input())

        if choice == 1:
            key_length = int(input("Enter AES key length (128 or 256): "))
            mode = AES.MODE_ECB if input("Enter mode (ECB or CFB): ") == 'ECB' else AES.MODE_CFB
            key = load_aes_key(key_length)
            data = pad(input("Enter data to encrypt: ").encode())
            measure_time(aes_encrypt, data, key, mode)

        elif choice == 2:
            key_length = int(input("Enter AES key length (128 or 256): "))
            mode = AES.MODE_ECB if input("Enter mode (ECB or CFB): ") == 'ECB' else AES.MODE_CFB
            key = load_aes_key(key_length)
            with open('aes_encrypted.bin', 'rb') as f:
                ciphertext = f.read()
            plaintext = measure_time(aes_decrypt, ciphertext, key, mode)
            print("Decrypted data:", plaintext.decode())

        elif choice == 3:
            private_key, public_key = load_rsa_keys()
            data = input("Enter data to encrypt: ").encode()
            measure_time(rsa_encrypt, data, public_key)

        elif choice == 4:
            private_key, public_key = load_rsa_keys()
            with open('rsa_encrypted.bin', 'rb') as f:
                ciphertext = f.read()
            plaintext = measure_time(rsa_decrypt, ciphertext, private_key)
            print("Decrypted data:", plaintext.decode())

        elif choice == 5:
            private_key, public_key = load_rsa_keys()
            data = input("Enter data to sign: ").encode()
            measure_time(rsa_sign, data, private_key)

        elif choice == 6:
            private_key, public_key = load_rsa_keys()
            data = input("Enter data to verify: ").encode()
            with open('rsa_signature.sig', 'rb') as f:
                signature = f.read()
            measure_time(rsa_verify, data, signature, public_key)

        elif choice == 7:
            data = input("Enter data to hash: ").encode()
            sha256_hash(data)

        elif choice == 8:
            break
        else:
            print("Invalid choice")

if __name__ == "__main__":
    generate_aes_key(128)
    generate_aes_key(256)
    generate_rsa_keys()
    main()

Choose an option:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
3
Enter data to encrypt: i am alvi
Time taken: 0.0027115345001220703 seconds
Choose an option:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
5
Enter data to sign: alvi
Time taken: 0.004503488540649414 seconds
Choose an option:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
7
Enter data to hash: doraemon
44de3ed43a38bbf73a9e4d5ea79f0b06ab29f5d003984a335cd3f79781c04e51
Choose an option:
1. AES Encryption
2. AES Decryption
3. RSA Encryption
4. RSA Decryption
5. RSA Signature
6. RSA Verification
7. SHA-256 Hashing
8. Exit
8
